In [17]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('sentiment-analysis-bert')
tokenizer = BertTokenizer.from_pretrained('sentiment-analysis-bert')

# Set the model to evaluation mode
model.eval()

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [32]:
def classifier(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Perform a forward pass through the model and get logits
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()

    # Create a list of labels
    labels = ["NEGATIVE", "POSITIVE"]

    # Format the output in the same structure as the pipeline
    results = []
    for prob in probabilities:
        results.append([{ "label": labels[i], "score": float(prob[i]) } for i in range(len(labels))])

    return results


In [33]:
texts = ["This movie was great!", "I did not enjoy the film at all."]
results = classifier(texts)
print(results)


[[{'label': 'NEGATIVE', 'score': 0.00436204019933939}, {'label': 'POSITIVE', 'score': 0.9956379532814026}], [{'label': 'NEGATIVE', 'score': 0.9912675023078918}, {'label': 'POSITIVE', 'score': 0.008732534013688564}]]


In [34]:
import shap

# Initialize the SHAP explainer
explainer = shap.Explainer(classifier, texts)

# Generate SHAP values for the first two texts
shap_values = explainer(texts[:2])

# Visualize SHAP explanations for the "POSITIVE" label (index 1)
shap.plots.text(shap_values[:, :, 1])


AttributeError: 'str' object has no attribute 'shape'